# Machine Learning project
- Validate exoplanets Kepler disposition with machine learning methods

In [423]:
%config Completer.use_jedi = False  # enable code auto-completion
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  #data visualization library
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix  # evaluation metrics

from sklearn.datasets import fetch_openml 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Dataset


In [424]:
df = pd.read_csv('q1_q17_dr24_koi_2023.09.19_04.32.41.csv')
df.columns =['KepID',"KOIName","KeplerName","ExoplanetArchiveDisposition",
"ExoplanetDispositionKepler","DispositionScore","NotTransit-LikeFalsePositiveFlag",
"koi_fpflag_ss","CentroidOffsetFalsePositiveFlag","EphemerisMatchIndicatesContaminationFalsePositiveFlag",
"OrbitalPeriod.[Days]","OrbitalPeriodErr1.[Days]","OrbitalPeriodErr2.[Days]",
"TransitEpoch-bk","TransitEpoch-bkErr1","TransitEpoch-bkErr2",
"ImpactParameter","ImpactParameterErr1","ImpactParameterErr2",
"TransitDuration.[Hours]","TransitDurationErr1.[Hours]","TransitDurationErr2.[Hours]",
"TransitDepth.[ppm]","TransitDepthErr1.[ppm]","TransitDepthErr2.[ppm]",
"PlanetaryRadius","PlanetaryRadiusErr1","PlanetaryRadiusErr2",
"EquilibriumTemperature.[K]","EquilibriumTemperatureErr1.[K]","EquilibriumTemperatureErr2.[K]",
"InsolationFlux","InsolationFluxErr1","InsolationFlux2",
"TransitSignalToNoise",
"TCEPlanetNumber","TCEDeliveryName",
"StellarEffectiveTemperature.[K]","StellarEffectiveTemperatureErr1.[K]","StellarEffectiveTemperatureErr2.[K]",
"StellarSurfaceGravity","StellarSurfaceGravityErr1","StellarSurfaceGravityErr2",
"StellarRadius","StellarRadiusErr1","StellarRadiusErr2",
"RA.[deg]","Dec.[deg]","KeplerMagnitude.[mag]"]


df.head()
#df['EquilibriumTemperatureErr2.[K]']

,KepID,KOIName,KeplerName,ExoplanetArchiveDisposition,ExoplanetDispositionKepler,DispositionScore,NotTransit-LikeFalsePositiveFlag,koi_fpflag_ss,CentroidOffsetFalsePositiveFlag,EphemerisMatchIndicatesContaminationFalsePositiveFlag,...,StellarEffectiveTemperatureErr2.[K],StellarSurfaceGravity,StellarSurfaceGravityErr1,StellarSurfaceGravityErr2,StellarRadius,StellarRadiusErr1,StellarRadiusErr2,RA.[deg],Dec.[deg],KeplerMagnitude.[mag]
0,11446443,K00001.01,Kepler-1 b,CONFIRMED,CANDIDATE,NaN,0,0,0,0,...,-50.0,4.455,0.025,-0.025,0.950,0.020,-0.020,286.80847,49.316399,11.338
1,10666592,K00002.01,Kepler-2 b,CONFIRMED,CANDIDATE,NaN,0,1,0,0,...,-80.0,4.021,0.011,-0.011,1.991,0.018,-0.018,292.24728,47.969521,10.463
2,10748390,K00003.01,Kepler-3 b,CONFIRMED,CANDIDATE,NaN,0,0,0,0,...,-91.0,4.590,0.011,-0.041,0.765,0.030,-0.022,297.70935,48.080853,9.174
3,3861595,K00004.01,Kepler-1658 b,CONFIRMED,CANDIDATE,NaN,0,1,0,0,...,-109.0,3.657,0.205,-0.107,2.992,0.469,-0.743,294.35654,38.947380,11.432
4,8554498,K00005.01,NaN,CANDIDATE,CANDIDATE,NaN,0,0,0,0,...,-75.0,4.003,0.011,-0.011,1.747,0.042,-0.042,289.73972,44.647419,11.665


# Modify dispositions
Modify KOI pdisposition and KOI disposition into binary form

In [425]:
df['ExoplanetCandidate'] = df['ExoplanetArchiveDisposition'].apply(lambda x: 1 if x == 'CANDIDATE' else 0)
df['ExoplanetConfirmed'] = df['ExoplanetDispositionKepler'].apply(lambda x: 2 if x == 'CONFIRMED' else 1 if x == 'CANDIDATE' else 0 )

In [426]:
df.drop(columns=['KeplerName','KOIName','EquilibriumTemperatureErr1.[K]', 'StellarEffectiveTemperatureErr2.[K]',
                 'KepID','ExoplanetArchiveDisposition','ExoplanetDispositionKepler',
                 'NotTransit-LikeFalsePositiveFlag','koi_fpflag_ss','CentroidOffsetFalsePositiveFlag',
                 'EphemerisMatchIndicatesContaminationFalsePositiveFlag','TCEDeliveryName', 'EquilibriumTemperatureErr2.[K]', 'DispositionScore'], inplace=True)
df.head()

,DispositionScore,OrbitalPeriod.[Days],OrbitalPeriodErr1.[Days],OrbitalPeriodErr2.[Days],TransitEpoch-bk,TransitEpoch-bkErr1,TransitEpoch-bkErr2,ImpactParameter,ImpactParameterErr1,ImpactParameterErr2,...,StellarSurfaceGravityErr1,StellarSurfaceGravityErr2,StellarRadius,StellarRadiusErr1,StellarRadiusErr2,RA.[deg],Dec.[deg],KeplerMagnitude.[mag],ExoplanetCandidate,ExoplanetConfirmed
0,NaN,2.470613,1.900000e-08,-1.900000e-08,122.763301,0.000006,-0.000006,0.8186,0.0003,-0.0006,...,0.025,-0.025,0.950,0.020,-0.020,286.80847,49.316399,11.338,0,1
1,NaN,2.204735,3.800000e-08,-3.800000e-08,121.358572,0.000014,-0.000014,0.0010,0.0111,-0.0010,...,0.011,-0.011,1.991,0.018,-0.018,292.24728,47.969521,10.463,0,1
2,NaN,4.887803,1.770000e-07,-1.770000e-07,124.813141,0.000028,-0.000028,0.0260,0.0486,-0.0260,...,0.011,-0.041,0.765,0.030,-0.022,297.70935,48.080853,9.174,0,1
3,NaN,3.849372,1.304000e-06,-1.304000e-06,157.526738,0.000269,-0.000269,0.9193,0.0059,-0.0071,...,0.205,-0.107,2.992,0.469,-0.743,294.35654,38.947380,11.432,0,1
4,NaN,4.780328,8.760000e-07,-8.760000e-07,132.974137,0.000152,-0.000152,0.9516,0.0011,-0.0023,...,0.011,-0.011,1.747,0.042,-0.042,289.73972,44.647419,11.665,1,1


In [427]:
df.isna().any()
df.shape


(7470, 38)

In [428]:
df_cleaned = df.dropna()

In [429]:
df_cleaned.shape

(0, 38)

# Features and labels

In [430]:
# Create the feature matrix X and label vector y. 
y = df_cleaned['ExoplanetCandidate'].to_numpy()
X = df_cleaned.drop(columns=['ExoplanetCandidate', 'ExoplanetConfirmed']).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=.40)

ValueError: With n_samples=0, test_size=0.4 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
clf = LogisticRegression(C=100, max_iter=1000, class_weight='balanced')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)



0.484